In [7]:
from collections import defaultdict
from dataclasses import dataclass, replace
import math
from typing import Dict

import numpy as np

In [16]:
@dataclass
class Bond:
    coupons: Dict[float, float]
    principal: float
    maturity: float
    
    @property
    def cashflows(self):
        c = defaultdict(lambda: 0, self.coupons)
        c[self.maturity] += self.principal
        return dict(c)
    
    def compute_price(self, y: float, t: float = 0) -> float:
        assert 0 <= t <= self.maturity
        return sum([c * math.exp(-y * (tc - t)) for tc, c in self.cashflows.items() if tc >= t]) 
    
    def compute_duration(self, y: float) -> float:
        B = self.compute_price(y)
        return sum([t * c * math.exp(- y * t) for t, c in self.cashflows.items()]) / B 
        
    def compute_convexity(self, y: float) -> float:
        B = self.compute_price(y)
        return sum([t ** 2 * c * math.exp(- y * t) for t, c in self.cashflows.items()]) / B 
                

In [28]:
def compute_price_impact(bond: Bond, y: float, delta_y: float) -> float:
    return bond.compute_price(y + delta_y) - bond.compute_price(y)

In [30]:
def estimate_price_impact(bond: Bond, y: float, delta_y: float, second_order: bool = False) -> float:
    B, D, C = bond.compute_price(y), bond.compute_duration(y), bond.compute_convexity(y)
    return - B * D * delta_y + int(second_order) * 0.5 * B * C * (delta_y ** 2)

In [35]:
bond = Bond(
    coupons={x: 5 for x in np.arange(0.5, 8.5, 0.5)},
    principal=100,
    maturity=8,
)

y = 0.13

print(f'B = {bond.compute_price(y):.03f}')
print(f'D = {bond.compute_duration(y):.03f}')
print(f'C = {bond.compute_convexity(y):.03f}')

for delta_y in (0.001, 0.01):

    print(f'\n*** Parallel shift of {delta_y * 100}% ***\n')
    print(f'True impact: {compute_price_impact(bond, y, delta_y):.4f}')
    print(f'First order impact: {estimate_price_impact(bond, y, delta_y):.4f}')
    print(f'Second order impact: {estimate_price_impact(bond, y, delta_y, second_order=True):.4f}')

B = 83.481
D = 5.447
C = 37.356

*** Parallel shift of 0.1% ***

True impact: -0.4531
First order impact: -0.4547
Second order impact: -0.4531

*** Parallel shift of 1.0% ***

True impact: -4.3947
First order impact: -4.5468
Second order impact: -4.3909
